# Temporary for debugging (initialize data and construct/initialize storage)

In [1]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [1]:
#!import "../Test/SpecificationsSetup"

<font size="+1">
Given the current status of the implementation, in order to display Actual reports it is necessary to set Actual format in the Args while to display Cashflow it is necessary to set Cashflow format.

In [1]:
var year          = 2021   ;
var month         = 3      ;
var reportingNode = "CH"   ;
var economicBasis = "L"    ;

In [1]:
var UniversePreview = await StartCalculatorAsync(reportingNode, year, month, economicBasis, Periodicity.Quarterly, ImportFormats.Cashflow);

In [1]:
var importStorage = UniversePreview.GetStorage();

# Import Preview

## Present Value

In [1]:
var identities = UniversePreview.GetScopes<GetIdentities>(importStorage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);

In [1]:
var pvs = UniversePreview.GetScopes<ComputeAllScopes>(identities, o => o.WithStorage(importStorage)).SelectMany(x => x.PvCurrent.Concat(x.PvLocked));

In [1]:
Report
    .ForObjects<IfrsVariable>(pvs
                        //.Where(x => x.DataNode == "DTR1.1")
    )
    .WithQuerySource(Workspace)
    .GroupColumnsBy(x => x.EconomicBasis)
    //.GroupColumnsBy(x => x.AmountType)
    .GroupColumnsBy(x => x.DataNode)
    .GroupRowsBy(x => x.Novelty)
    .GroupRowsBy(x => x.AocType)
    //VariableType
    //.GroupColumnsBy(x => x.DataNode)
    //.GroupColumnsBy(x => x.Novelty)
    .WithGridOptionsForIfrsVariable(720)
    .ToReport()

## Risk Adjustment

In [1]:
var ras = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.RaCurrent.Concat(x.RaLocked));

In [1]:
Report
    .ForObjects<IfrsVariable>(ras)
    .WithQuerySource(Workspace)
    .GroupColumnsBy(x => x.EconomicBasis)
    //.GroupRowsBy(x => x.DataNode)
    .GroupRowsBy(x => x.Novelty)
    .GroupRowsBy(x => x.AocType)
    //VariableType
    //.GroupColumnsBy(x => x.DataNode)
    //.GroupColumnsBy(x => x.Novelty)
    .WithGridOptionsForIfrsVariable()
    .ToReport()

## Actuals

In [1]:
var actuals = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.Actual);

In [1]:
Report
    .ForObjects<IfrsVariable>(actuals)
    .WithQuerySource(Workspace)
    .GroupRowsBy(x => x.AmountType)
    //.GroupColumnsBy(x => x.DataNode)
    .WithGridOptionsForIfrsVariable(550)
    .ToReport()

## Advance, Overdue Actuals

In [1]:
var aoActuals = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.AdvanceActual.Concat(x.OverdueActual));

In [1]:
Report
    .ForObjects<IfrsVariable>(aoActuals)
    .WithQuerySource(Workspace)
    .GroupColumnsBy(x => x.DataNode)
    .GroupColumnsBy(x => x.EstimateType)
    .GroupColumnsBy(x => x.AmountType)
    .GroupRowsBy(x => x.AocType)
    .WithGridOptionsForIfrsVariable(300)
    .ToReport()

## Deferrable Actuals

In [1]:
var deferrableActuals = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.DeferrableActual);

In [1]:
Report
    .ForObjects<IfrsVariable>(deferrableActuals)
    .WithQuerySource(Workspace)
    .GroupColumnsBy(x => x.DataNode)
    .GroupRowsBy(x => x.AocType)
    .WithGridOptionsForIfrsVariable(350)
    .ToReport()

## Csm Lc LoReCo

In [1]:
var csm = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.Csms);
var loss = UniversePreview.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.Loss);
var csmLcLoReCo = Enumerable.Empty<IfrsVariable>().Concat(csm).Concat(loss);

In [1]:
Report
    .ForObjects<IfrsVariable>(csmLcLoReCo
                            //.Where(x => x.DataNode == "DT1.1")
                            )
    .WithQuerySource(Workspace)
    .GroupColumnsBy(x => x.DataNode)
    .GroupColumnsBy(x => x.EstimateType)
    .GroupRowsBy(x => x.Novelty)
    .GroupRowsBy(x => x.AocType)
    .WithGridOptionsForIfrsVariable()
    .ToReport()